# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/maxcarey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/maxcarey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 60, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/51 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/60 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/93 [00:00<?, ?it/s]

Property 'summary' already exists in node '037329'. Skipping!
Property 'summary' already exists in node '9d1814'. Skipping!
Property 'summary' already exists in node '39f495'. Skipping!
Property 'summary' already exists in node 'adc8f5'. Skipping!
Property 'summary' already exists in node '77f539'. Skipping!
Property 'summary' already exists in node 'e01d92'. Skipping!
Property 'summary' already exists in node '7e1e7f'. Skipping!
Property 'summary' already exists in node '8761a9'. Skipping!
Property 'summary' already exists in node 'e82bb6'. Skipping!
Property 'summary' already exists in node 'f6a4c7'. Skipping!
Property 'summary' already exists in node 'bf4177'. Skipping!
Property 'summary' already exists in node 'fba047'. Skipping!
Property 'summary' already exists in node '90fb54'. Skipping!
Property 'summary' already exists in node '520195'. Skipping!
Property 'summary' already exists in node '6fdc14'. Skipping!
Property 'summary' already exists in node '09378b'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/129 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '037329'. Skipping!
Property 'summary_embedding' already exists in node '9d1814'. Skipping!
Property 'summary_embedding' already exists in node '39f495'. Skipping!
Property 'summary_embedding' already exists in node '7e1e7f'. Skipping!
Property 'summary_embedding' already exists in node 'e01d92'. Skipping!
Property 'summary_embedding' already exists in node 'e82bb6'. Skipping!
Property 'summary_embedding' already exists in node 'adc8f5'. Skipping!
Property 'summary_embedding' already exists in node 'f6a4c7'. Skipping!
Property 'summary_embedding' already exists in node 'bf4177'. Skipping!
Property 'summary_embedding' already exists in node 'fba047'. Skipping!
Property 'summary_embedding' already exists in node '8761a9'. Skipping!
Property 'summary_embedding' already exists in node '77f539'. Skipping!
Property 'summary_embedding' already exists in node '520195'. Skipping!
Property 'summary_embedding' already exists in node '6fdc14'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 120, relationships: 4359)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 120, relationships: 4359)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:



What are the three types of query synthesizers doing? Describe each one in simple terms.

SingleHopSpecificQuerySynthesizer: Generates straightforward, single-step questions that target specific facts or details from the data. For example, "What is the interest rate for Direct Subsidized Loans?"

MultiHopAbstractQuerySynthesizer: Creates questions that require connecting multiple pieces of information but focus on higher-level concepts or patterns. For example, "How does the FAFSA verification process impact loan disbursement timing?"

MultiHopSpecificQuerySynthesizer: Produces questions that need multiple specific pieces of information to be combined for a complete answer. For example, "What are the eligibility requirements for both Direct Subsidized and Unsubsidized loans, and how do they differ?"



Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How are payment periods defined within the con...,"[Chapter 1 Academic Years, Academic Calendars,...",Payment periods are mentioned alongside academ...,single_hop_specifc_query_synthesizer
1,What does the regulation 34 CFR 668.3(b) speci...,[Regulatory Citations Academic year minimums: ...,The regulation 34 CFR 668.3(b) pertains to wee...,single_hop_specifc_query_synthesizer
2,What does Chapter 3 cover regarding clinical w...,[Inclusion of Clinical Work in a Standard Term...,Chapter 3 discusses the inclusion of clinical ...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in academic ...,[Non-Term Characteristics A program that measu...,A program that measures progress in clock hour...,single_hop_specifc_query_synthesizer
4,What is a Direct Loan?,[both the credit or clock hours and the weeks ...,A Direct Loan is a type of federal student loa...,single_hop_specifc_query_synthesizer
5,How do the different academic year definitions...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The different academic year definitions for va...,multi_hop_abstract_query_synthesizer
6,How does the variation in academic year defini...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that schools may define d...,multi_hop_abstract_query_synthesizer
7,How does the inclusion of clinical work that o...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work that overlaps s...,multi_hop_abstract_query_synthesizer
8,How does the inclusion of clinical work in a s...,[<1-hop>\n\nInclusion of Clinical Work in a St...,"According to the information in Volume 8, clin...",multi_hop_specific_query_synthesizer
9,How do the definitions of academic years and i...,[<1-hop>\n\nboth the credit or clock hours and...,The definitions of academic years and instruct...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'f1ef88'. Skipping!
Property 'summary' already exists in node '3837ca'. Skipping!
Property 'summary' already exists in node '97c011'. Skipping!
Property 'summary' already exists in node '4aa89b'. Skipping!
Property 'summary' already exists in node '32d81b'. Skipping!
Property 'summary' already exists in node 'a309af'. Skipping!
Property 'summary' already exists in node 'a05dbd'. Skipping!
Property 'summary' already exists in node '79eb29'. Skipping!
Property 'summary' already exists in node '5866cf'. Skipping!
Property 'summary' already exists in node 'e59432'. Skipping!
Property 'summary' already exists in node '79ebac'. Skipping!
Property 'summary' already exists in node '3e02df'. Skipping!
Property 'summary' already exists in node 'd0bed4'. Skipping!
Property 'summary' already exists in node '1d2559'. Skipping!


...

Property 'summary_embedding' already exists in node 'd0bed4'. Skipping!
Property 'summary_embedding' already exists in node 'f1ef88'. Skipping!
Property 'summary_embedding' already exists in node '5866cf'. Skipping!
Property 'summary_embedding' already exists in node '97c011'. Skipping!
Property 'summary_embedding' already exists in node '4aa89b'. Skipping!
Property 'summary_embedding' already exists in node '79ebac'. Skipping!
Property 'summary_embedding' already exists in node '3837ca'. Skipping!
Property 'summary_embedding' already exists in node 'a309af'. Skipping!
Property 'summary_embedding' already exists in node '32d81b'. Skipping!
Property 'summary_embedding' already exists in node '79eb29'. Skipping!
Property 'summary_embedding' already exists in node '3e02df'. Skipping!
Property 'summary_embedding' already exists in node 'a05dbd'. Skipping!
Property 'summary_embedding' already exists in node '1d2559'. Skipping!
Property 'summary_embedding' already exists in node 'e59432'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Could you please explain what is encompassed b...,"[Chapter 1 Academic Years, Academic Calendars,...","In the provided context, 'Volume 2' refers to ...",single_hop_specifc_query_synthesizer
1,What does the regulation 34 CFR 668.3(b) speci...,[Regulatory Citations Academic year minimums: ...,Regulatory citation 34 CFR 668.3(b) pertains t...,single_hop_specifc_query_synthesizer
2,"Chapter 3 include clinical work or not, and ho...",[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in relation ...,[Non-Term Characteristics A program that measu...,Non-Term Characteristics refer to programs tha...,single_hop_specifc_query_synthesizer
4,How do policy requirements for clinical work i...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work conduc...,multi_hop_abstract_query_synthesizer
5,how disbursement timing work for financial aid...,[<1-hop>\n\nboth the credit or clock hours and...,"In programs with clock hours, disbursement of ...",multi_hop_abstract_query_synthesizer
6,How do the program requirements under 34 CFR 6...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The program requirements outlined in 34 CFR 66...,multi_hop_abstract_query_synthesizer
7,so tell me how academic years and regulatory c...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each academic progra...,multi_hop_abstract_query_synthesizer
8,How do Chapters 2 and 3 relate to the inclusio...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Chapter 2 discusses the criteria for including...,multi_hop_specific_query_synthesizer
9,Wht appendix A and B tel about disbursement ti...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [20]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data 2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [21]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [22]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [24]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [25]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [26]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [27]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [29]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [30]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct PLUS Loan or student Federal PLUS Loan  \n- Subsidized Federal Stafford Loans  \n- Unsubsidized Federal Stafford Loans  \n- Federal SLS Loans  \n- Federal PLUS Loans  \n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct Consolidation Loans  \n- Federal Consolidation Loans  \n\nAdditionally, it is noted that Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the FFEL Program before July 1, 2010, with no new loans under FFEL after that. Direct Subsidized and Unsubsidized Loans and Direct PLUS Loans are still available.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [31]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [32]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

#### ✅ Under the hood, these evaluators use LangChain/LangSmith's evaluation framework and an LLM (configured as eval_llm) to assess responses:

- `qa_evaluator`: Acts like a unit test for question-answering. It uses the LLM to check if the model's response correctly answers the user's question, similar to how we validate function outputs match expected results.

- `labeled_helpfulness_evaluator`: Similar to an integration test that validates multiple aspects through the LLM:
  1. Takes the model's output ("prediction")
  2. Compares it against a known correct answer ("reference") 
  3. Considers the original question ("input")
  4. Evaluates if the response was actually helpful to the user
  This is like testing if an API response not only returns valid data but is also useful for the client.

- `empathy_evaluator`: Tests the UX/tone of responses using the LLM. Just as we want friendly error messages in our applications, this evaluator checks if the AI's responses are empathetic and make users feel heard, rather than being overly technical or cold.

Each evaluator leverages LangSmith's LangChainStringEvaluator class, which provides a standardized way to evaluate model outputs using customizable criteria and LLM-based assessment.



## LangSmith Evaluation

In [33]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'brief-sign-90' at:
https://smith.langchain.com/o/add79bbe-c927-4cfa-8dbd-3fe101dd7548/datasets/351985dc-4a42-4780-8075-0a3fe11a5337/compare?selectedSessions=d4ff12f5-7a61-48b7-a96d-7e38ee0aef1e




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 8 and Volume 7 what about disbursement?,Based on the provided context:\n\n- Volume 7 (...,None,"In Volume 8, it says that clinical work outsid...",1,0,0,5.850575,23184268-039d-4a95-906b-9ee9399b3636,06bb1edc-ee5f-4a70-8c6a-ea5ef383bade
1,How do Volume 2 and Volume 8 relate to disburs...,I don't know.,None,Volume 2 discusses the requirements for academ...,0,0,0,1.126535,aa6e3527-42ae-4a3e-93f8-2fa17fc3ae48,bef468cd-8fa9-4ef3-80fe-3fbed1065277
2,Wht appendix A and B tel about disbursement ti...,"Based on the provided context, Appendix B of V...",None,Appendix B provides detailed guidance on disbu...,0,0,0,10.673446,226dfece-5f95-416e-b1f5-95b8ae112064,f0b2bbbc-0d25-4b42-a968-899cac97640e
3,How do Chapters 2 and 3 relate to the inclusio...,I don't know.,None,Chapter 2 discusses the criteria for including...,0,0,0,0.743086,39e87aa5-19fb-4e15-a9c0-996c277452a7,3dbf789d-39b9-4c26-8475-11bcd4b76332
4,so tell me how academic years and regulatory c...,"Based on the provided context, academic years ...",None,The context explains that each academic progra...,1,1,0,8.641882,41edc792-be59-4bef-97bd-fa71964c890b,1e7657b2-3676-40cc-a1b5-fd2d9f08d410
5,How do the program requirements under 34 CFR 6...,I don't know.,None,The program requirements outlined in 34 CFR 66...,0,0,0,0.973859,0c2f94f7-24ba-449c-98f8-30f0643cb3c3,7743ff65-2da3-474b-8df7-7a161a90ae66
6,how disbursement timing work for financial aid...,Based on the provided context:\n\n**Disburseme...,None,"In programs with clock hours, disbursement of ...",1,1,0,18.088894,8dfa675f-fc09-4c3b-a317-4c93c4e5e931,04e9364c-3c85-4f42-97de-787bb64d84eb
7,How do policy requirements for clinical work i...,"Based on the provided context, clinical work c...",None,The context explains that clinical work conduc...,1,1,0,6.927937,3af53b6d-4011-43d7-ac33-2c4bc3e11ed0,a9f77860-95db-4dea-802a-f7b64b9c5c58
8,What are Non-Term Characteristics in relation ...,Non-Term Characteristics refer to programs tha...,None,Non-Term Characteristics refer to programs tha...,1,1,0,8.066107,97960aaf-c37e-47d1-818e-b1c711f9d0e8,8a39a889-47d2-4239-b0d4-01c7a0e15c30
9,"Chapter 3 include clinical work or not, and ho...",Chapter 3 includes clinical work when the clin...,None,Inclusion of clinical work in a standard term ...,1,0,0,4.337231,ee4ab1de-c982-4039-9fe7-fa95f5db102b,dfb08603-b905-417a-8491-0e42f07c98de


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [34]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [35]:
rag_documents = docs

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

# ✅

Modifying chunk size affects performance in several ways:

1. Larger chunks provide more context in each segment, allowing the model to better understand relationships and answer complex questions that require more contextual information

2. However, larger chunks also mean fewer total chunks, which could reduce granularity in retrieval and potentially miss relevant information if it's split across chunks

3. The chunk overlap helps maintain continuity between segments, but needs to be balanced - too much overlap increases storage and processing requirements while too little could miss cross-chunk relationships

4. The optimal chunk size depends on the nature of the content and types of questions being asked - technical content may benefit from larger chunks to capture complete concepts


In [37]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

# #### ✅

Modifying the embedding model can impact performance in several ways:

1. Different embedding models have varying capabilities in capturing semantic meaning and relationships between words/concepts

2. More advanced models (like text-embedding-3-large) generally produce higher quality embeddings that better represent the nuances and context of the text

3. The dimensionality and quality of embeddings affects how well similar documents can be retrieved during the search process

4. The choice of embedding model needs to balance performance requirements with computational costs and latency

In [38]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [39]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [40]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [41]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question! From the information provided in the context, it seems there are several types of federal student loans available:\n\n1. **Direct Subsidized Loans** – These are loans for students with financial need, where the government pays the interest while the student is in school at least half-time, during the grace period, and during deferment periods.\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need, but interest accrues during all periods.\n\n3. **Direct PLUS Loans** – These are loans that parents of dependent students can take out to help pay for their child's cost of attendance, assuming the parent meets eligibility requirements. Graduate or professional students can also apply for Direct PLUS Loans.\n\nAdditionally, if a dependent student's parent is unable to obtain a Direct PLUS Loan, the student may be eligible for increased amounts of Direct Unsubsidized Loans to help cover costs.\n\nI hope this helps

Finally, we can evaluate the new chain on the same test set!

In [42]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'crazy-interest-87' at:
https://smith.langchain.com/o/add79bbe-c927-4cfa-8dbd-3fe101dd7548/datasets/351985dc-4a42-4780-8075-0a3fe11a5337/compare?selectedSessions=41acbff5-90d2-4a18-9d2e-d98f186adad4




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 8 and Volume 7 what about disbursement?,Thank you for your question—I understand you'd...,None,"In Volume 8, it says that clinical work outsid...",0,0,1,4.463199,23184268-039d-4a95-906b-9ee9399b3636,01d891c9-7fc4-491e-ba0b-200eac6a44c4
1,How do Volume 2 and Volume 8 relate to disburs...,Thank you for your thoughtful question. I can ...,None,Volume 2 discusses the requirements for academ...,0,0,1,4.257032,aa6e3527-42ae-4a3e-93f8-2fa17fc3ae48,26c78251-d627-461c-8205-67b98282b928
2,Wht appendix A and B tel about disbursement ti...,Thank you for your question. Based on the cont...,None,Appendix B provides detailed guidance on disbu...,0,0,1,3.648459,226dfece-5f95-416e-b1f5-95b8ae112064,d43abf3d-db99-477d-8ded-adf1fc520a22
3,How do Chapters 2 and 3 relate to the inclusio...,Thank you for your thoughtful question. Based ...,None,Chapter 2 discusses the criteria for including...,1,1,1,3.099247,39e87aa5-19fb-4e15-a9c0-996c277452a7,5b0f2133-2698-467f-9e73-ae743fec0e06
4,so tell me how academic years and regulatory c...,Thank you for your thoughtful question. I unde...,None,The context explains that each academic progra...,1,1,1,7.890160,41edc792-be59-4bef-97bd-fa71964c890b,98c7d79b-9b8a-49fb-921d-d7b3adb61fd6
5,How do the program requirements under 34 CFR 6...,Thank you for your thoughtful question. Based ...,None,The program requirements outlined in 34 CFR 66...,0,0,1,2.961466,0c2f94f7-24ba-449c-98f8-30f0643cb3c3,2ac8cd93-35d1-4109-8369-e45d6083e957
6,how disbursement timing work for financial aid...,Thank you for your thoughtful question. I unde...,None,"In programs with clock hours, disbursement of ...",1,1,1,12.701606,8dfa675f-fc09-4c3b-a317-4c93c4e5e931,a6b2f994-a371-4c81-8e0f-c6e546258614
7,How do policy requirements for clinical work i...,Thank you for your thoughtful question. Based ...,None,The context explains that clinical work conduc...,0,0,1,5.142137,3af53b6d-4011-43d7-ac33-2c4bc3e11ed0,cdee5c86-29d7-453d-a105-ed2c6cacc7cf
8,What are Non-Term Characteristics in relation ...,Thank you for your thoughtful question. It sou...,None,Non-Term Characteristics refer to programs tha...,1,1,1,6.416600,97960aaf-c37e-47d1-818e-b1c711f9d0e8,4714cf06-f229-4b9b-9c02-bc63b74948ec
9,"Chapter 3 include clinical work or not, and ho...",Thank you for your thoughtful question. Based ...,None,Inclusion of clinical work in a standard term ...,1,1,1,5.627140,ee4ab1de-c982-4039-9fe7-fa95f5db102b,d9457359-9cb6-4198-805d-817d9da029bd


#### 🏗️ Activity #3:


![Screenshot](screenshot.png)



It looks like our empathy socre went way up, every response became empathetic, this is no surprise. Interestingly, we may have sacrified some correctness for empathy, though